In [6]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder.appName("Case3").getOrCreate()

In [9]:
sc = spark.sparkContext

In [10]:
sc

<SparkContext master=local[*] appName=Case3>

In [11]:
df=spark.read.format('csv').option('header',True).load('file:///C:/Users/HP/Documents/subho/EmployeeDatabase.csv')

In [12]:
df.show(2)

+-----------+-------+------------+--------------+--------------------+------------------+------------+
|Employee_id| Skills|Joining_Date|Num_of_project|Next_Gen_eligibility|eligible_promotion|Careel_Level|
+-----------+-------+------------+--------------+--------------------+------------------+------------+
|   10000001|    aws|   26Jun2005|             4|                  no|                no|          L1|
|   10000002|pyspark|   03Feb2007|             9|                  no|                no|          L4|
+-----------+-------+------------+--------------+--------------------+------------------+------------+
only showing top 2 rows



In [13]:
from pyspark.sql import functions as F

In [ ]:
## add a column Next_Gen_eligibility if skill in 'pyspark', 'aws', 'scala'

In [14]:
df=df.withColumn('Next_Gen_eligibility', \
                 F.when(F.col('Skills').isin(['pyspark', 'aws', 'scala']) ,\
                        F.lit('yes')).otherwise(F.lit('no')))

In [42]:
df.show(2)

+-----------+-------+------------+--------------+--------------------+------------------+------------+
|Employee_id| Skills|Joining_Date|Num_of_project|Next_Gen_eligibility|eligible_promotion|Careel_Level|
+-----------+-------+------------+--------------+--------------------+------------------+------------+
|   10000001|    aws|   26Jun2005|             4|                 yes|                no|          L1|
|   10000002|pyspark|   03Feb2007|             9|                 yes|                no|          L4|
+-----------+-------+------------+--------------+--------------------+------------------+------------+
only showing top 2 rows



In [15]:
# assign base_salary based on career_level

In [43]:
from pyspark.sql.functions import udf

@udf
def assignSal(lvl):
    if lvl == 'L1':
        return 9999
    elif lvl == 'L2':
        return 8888
    elif lvl == 'L3':
        return 7777
    elif lvl == 'L4':
        return 6666

In [44]:
df = df.withColumn('base_salary', assignSal(F.col('Careel_Level')))

In [45]:
df.show(2)

+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+
|Employee_id| Skills|Joining_Date|Num_of_project|Next_Gen_eligibility|eligible_promotion|Careel_Level|base_salary|
+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+
|   10000001|    aws|   26Jun2005|             4|                 yes|                no|          L1|       9999|
|   10000002|pyspark|   03Feb2007|             9|                 yes|                no|          L4|       6666|
+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+
only showing top 2 rows



In [46]:
df.show(2)

+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+
|Employee_id| Skills|Joining_Date|Num_of_project|Next_Gen_eligibility|eligible_promotion|Careel_Level|base_salary|
+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+
|   10000001|    aws|   26Jun2005|             4|                 yes|                no|          L1|       9999|
|   10000002|pyspark|   03Feb2007|             9|                 yes|                no|          L4|       6666|
+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+
only showing top 2 rows



In [48]:
import datetime

In [49]:
@udf
def calExpDays(dt):
    dtFmt = datetime.datetime.strptime(dt, '%d%b%Y')
    date_diff = datetime.datetime.now() - dtFmt
    if date_diff.days > 0:
        return date_diff.days
    return 0

In [50]:
df = df.withColumn('numOfDays', calExpDays('Joining_Date'))

In [ ]:
#actual_salary = base_salary*3.4 + numOfdays/365
a=3.14

In [51]:
df = df.withColumn('actual_salary', F.col('base_salary') * F.lit(3.14) +  F.col('numOfDays')/F.lit(365))

In [52]:
df.show(2)

+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+---------+------------------+
|Employee_id| Skills|Joining_Date|Num_of_project|Next_Gen_eligibility|eligible_promotion|Careel_Level|base_salary|numOfDays|     actual_salary|
+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+---------+------------------+
|   10000001|    aws|   26Jun2005|             4|                 yes|                no|          L1|       9999|     5308|31411.402465753425|
|   10000002|pyspark|   03Feb2007|             9|                 yes|                no|          L4|       6666|     4721|20944.174246575345|
+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+---------+------------------+
only showing top 2 rows



In [53]:
df.filter().show(2)F.col('actual_salary')>1000

+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+---------+------------------+
|Employee_id| Skills|Joining_Date|Num_of_project|Next_Gen_eligibility|eligible_promotion|Careel_Level|base_salary|numOfDays|     actual_salary|
+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+---------+------------------+
|   10000001|    aws|   26Jun2005|             4|                 yes|                no|          L1|       9999|     5308|31411.402465753425|
|   10000002|pyspark|   03Feb2007|             9|                 yes|                no|          L4|       6666|     4721|20944.174246575345|
+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+---------+------------------+
only showing top 2 rows



In [54]:
df.filter(~(F.col('actual_salary')>24499)).show(2)

+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+---------+------------------+
|Employee_id| Skills|Joining_Date|Num_of_project|Next_Gen_eligibility|eligible_promotion|Careel_Level|base_salary|numOfDays|     actual_salary|
+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+---------+------------------+
|   10000002|pyspark|   03Feb2007|             9|                 yes|                no|          L4|       6666|     4721|20944.174246575345|
|   10000003| python|   28Mar2013|             6|                  no|                no|          L3|       7777|     2476|24426.563561643838|
+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+---------+------------------+
only showing top 2 rows



In [55]:
df = df.withColumn('eligible_promotion',\
              F.when((F.col('numOfDays') > 1000 ) & (F.col('Next_Gen_eligibility')=='yes'), \
                     F.lit('yes'))\
                   .otherwise(  F.when(F.col('numOfDays') > 5000 , F.lit('yes'))\
                              .otherwise(F.lit('no'))   )  )

In [56]:
df.show(2)

+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+---------+------------------+
|Employee_id| Skills|Joining_Date|Num_of_project|Next_Gen_eligibility|eligible_promotion|Careel_Level|base_salary|numOfDays|     actual_salary|
+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+---------+------------------+
|   10000001|    aws|   26Jun2005|             4|                 yes|               yes|          L1|       9999|     5308|31411.402465753425|
|   10000002|pyspark|   03Feb2007|             9|                 yes|               yes|          L4|       6666|     4721|20944.174246575345|
+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+---------+------------------+
only showing top 2 rows



In [66]:
df2 = df.groupby('Careel_Level').avg('actual_salary').toDF('Careel_Level', 'avg_actual_salary')\
        .orderBy(df.Careel_Level.asc())

In [67]:
df2.show()

+------------+------------------+
|Careel_Level| avg_actual_salary|
+------------+------------------+
|          L1|31403.909970526074|
|          L2|27915.379353907374|
|          L3|24426.985622203323|
|          L4|20938.097919996377|
+------------+------------------+



In [71]:
df3 = df.join(df2, df.Careel_Level == df2.Careel_Level, 'left')

In [73]:
df3.show(2)

+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+---------+------------------+------------+------------------+
|Employee_id| Skills|Joining_Date|Num_of_project|Next_Gen_eligibility|eligible_promotion|Careel_Level|base_salary|numOfDays|     actual_salary|Careel_Level| avg_actual_salary|
+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+---------+------------------+------------+------------------+
|   10000001|    aws|   26Jun2005|             4|                 yes|               yes|          L1|       9999|     5308|31411.402465753425|          L1|31403.909970526074|
|   10000002|pyspark|   03Feb2007|             9|                 yes|               yes|          L4|       6666|     4721|20944.174246575345|          L4|20938.097919996377|
+-----------+-------+------------+--------------+--------------------+------------------+------------+-----------+------